In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import flopy

### Load the existing MODFLOW-NWT model

In [ ]:
org_nwt_ws = os.path.join("..", "..", "data", "freyberg_nwt")
assert os.path.exists(org_nwt_ws)

In [ ]:
nam_file = "freyberg.nam"
m_nwt = flopy.modflow.Modflow.load(nam_file, model_ws=org_nwt_ws, exe_name="mfnwt", forgive=False, check=False)

### Create the MODFLOW 6 model from the MODFLOW-NWT data in the flopy model object

In [ ]:
mf6_ws = "temp"
name = nam_file.replace(".nam", "")
sim = flopy.mf6.MFSimulation(sim_name=name, exe_name="mf6", sim_ws=mf6_ws)

In [ ]:
perioddata = list(zip(m_nwt.dis.perlen.array, m_nwt.dis.nstp.array, m_nwt.dis.tsmult))
perioddata

In [ ]:
tdis = flopy.mf6.ModflowTdis(sim, perioddata=perioddata, nper=m_nwt.nper)

In [ ]:
m_mf6 = flopy.mf6.ModflowGwf(sim, modelname=name, exe_name="mf6")

In [ ]:
ims = flopy.mf6.ModflowIms(sim)

In [ ]:
dis = flopy.mf6.ModflowGwfdis(m_mf6, nrow=m_nwt.nrow, ncol=m_nwt.ncol, nlay=m_nwt.nlay,
                              delr=m_nwt.dis.delr.array, delc=m_nwt.dis.delc.array, 
                              top=m_nwt.dis.top.array, botm=m_nwt.dis.botm.array,
                              idomain=m_nwt.bas6.ibound.array)

In [ ]:
wel_spd = {}
for kper in range(m_nwt.nper):
    df = pd.DataFrame.from_records(m_nwt.wel.stress_period_data[kper])
    df.loc[:,"kij"] = df.apply(lambda x: (int(x.k), int(x.i), int(x.j)), axis=1)
    wel_spd[kper] = [df.loc[i, ["kij","flux"]].to_list() for i in range(df.shape[0])]

In [ ]:
wel_spd[0]

In [ ]:
wel = flopy.mf6.ModflowGwfwel(m_mf6,stress_period_data=wel_spd)

In [ ]:
riv_j = 14
riv_locs = [(0,i,riv_j) for i in range(m_nwt.nrow)]
stages = np.linspace(m_nwt.dis.top[0,riv_j], m_nwt.dis.top[-1,riv_j], m_nwt.nrow)
conds = np.zeros((m_nwt.nrow)) + 100
rbots = stages - 2
riv_data = list(zip(riv_locs, stages, conds, rbots))

In [ ]:
riv = flopy.mf6.ModflowGwfriv(m_mf6, stress_period_data=riv_data)

In [ ]:
npf = flopy.mf6.ModflowGwfnpf(m_mf6, k=m_nwt.upw.hk.array, k33=m_nwt.upw.vka.array)

In [ ]:
sto = flopy.mf6.ModflowGwfsto(m_mf6, ss=m_nwt.upw.ss.array, sy=m_nwt.upw.sy.array)

In [ ]:
ic = flopy.mf6.ModflowGwfic(m_mf6, strt=m_nwt.bas6.strt.array)

In [ ]:
chd_data = [[(m_nwt.nlay-1, m_nwt.nrow-1,j), riv_data[-1][1]] for j in range(m_nwt.ncol) if m_nwt.bas6.ibound.array[-1, -1, j] != 0]
chd_data

In [ ]:
chd = flopy.mf6.ModflowGwfchd(m_mf6, stress_period_data=chd_data)

In [ ]:
rch = flopy.mf6.ModflowGwfrcha(m_mf6, recharge = {kper: m_nwt.rch.rech[kper].array for kper in range(m_nwt.nper)})

In [ ]:
oc = flopy.mf6.ModflowGwfoc(m_mf6, head_filerecord=name + ".hds", saverecord=(["head", "all"]), printrecord=(["budget", "all"]))

### Write the MODFLOW 6 model files

In [ ]:
sim.write_simulation()

### Run the MODFLOW 6 model

In [ ]:
success = sim.run_simulation()

### Post process the MODFLOW 6 results

Simulated heads

In [ ]:
hds = m_mf6.output.head()

In [ ]:
hds.plot(masked_values=[1e30]);

MODFLOW 6 water budget from the listing file

In [ ]:
lst = m_mf6.output.list()
flx, vol = lst.get_dataframes(diff=True)

In [ ]:
flx.plot(kind="bar",figsize=(10,10));